In [1]:
import os
import pandas as pd

In [2]:
my_dir = "D:/Thesis/Enveloppes/"

In [20]:
res = []
for file_name in os.listdir(my_dir):
    
    
    df = pd.read_excel(os.path.join(my_dir, file_name))
    print(file_name, "is being processed \n")
    
    dfs = []
    for unique_val in df['Cap2'].unique():
        df_filtered = df[(df["Cap2"] == unique_val)]
        
        print("Captor ", unique_val, "is being processed")
        print("______________________________________ \n")
        
        groupes = []
        groupe_actuel = []
        seuil = 0.1
        cutt_off = 2

        for i in range(len(df_filtered) - 1):
            current_row = df_filtered.iloc[i]
            next_row = df_filtered.iloc[i + 1]

            if not groupe_actuel:
                groupe_actuel.append((current_row['time_min'], current_row['mean_dist']))
            else:
                # Vérification de la proximité de la distance et de la consécutivité du temps
                if abs(groupe_actuel[-1][1] - current_row['mean_dist']) <= seuil and next_row['time_min'] - current_row['time_min'] == 1:
                    groupe_actuel.append((current_row['time_min'], current_row['mean_dist']))
                else:
                    groupes.append(groupe_actuel)
                    groupe_actuel = [(current_row['time_min'], current_row['mean_dist'])]

        if groupe_actuel:
            groupes.append(groupe_actuel)

        # Filtrer les groupes selon la condition de distance
        groupes_filtrés = [groupe for groupe in groupes if all(d <= cutt_off for _, d in groupe)]
        top3_groupes = sorted(groupes_filtrés, key=len, reverse=True)[:3]
        # Affichage des groupes filtrés
        #for idx, groupe in enumerate(top3_groupes):
        #    print(f'Groupe {idx + 1} :')
        #    print('Temps    Distances')
        #    for t, d in groupe:
        #        print(f'{t}       {d}')
        #    print()
            
        data = []
        intervals = []        

        
        if (len(top3_groupes)> 0):
        
            # Attribuer un ID unique à chaque groupe du top 3
            for group_id, groupe in enumerate(top3_groupes, start=1):

                start_time = groupe[0][0]
                end_time = groupe[-1][0]
                intervals.append({'Groupe_ID': group_id, 'Debut': start_time, 'Fin': end_time})
                for point in groupe:
                        # Pour chaque observation dans le groupe, nous stockons l'ID du groupe, le temps et la distance
                        data.append({'Groupe_ID': group_id, 'Temps': point[0], 'Distance': point[1]})         

            # Convertir les données et les intervalles en DataFrames
            df_top3 = pd.DataFrame(data)
            df_intervals = pd.DataFrame(intervals)

            # Calculer la durée et la médiane pour chaque groupe
            df_intervals['Duree_min'] = df_intervals['Fin'] - df_intervals['Debut'] + 1
            #df_intervals['Duree_min'] = df_intervals['Duree_sec'] / 60
            df_intervals['Distance Mediane'] = df_intervals['Groupe_ID'].apply(lambda x: df_top3[df_top3['Groupe_ID'] == x]['Distance'].median())

            # Vous pouvez fusionner le DataFrame df_top3 avec df_intervals sur Groupe_ID pour avoir toutes les informations dans un seul DataFrame
            df_merged = pd.merge(df_top3, df_intervals, on="Groupe_ID")

            summary = df_merged[["Groupe_ID", "Duree_min", "Distance Mediane"]].drop_duplicates()
            
            print(summary, "\n")
            
            new_df = pd.DataFrame()

            # Boucler sur les lignes de df
            for index, row in summary.iterrows():

                # Créer les nouveaux noms de colonnes
                col1 = f"Duree_min_{row['Groupe_ID']}"
                col2 = f"Distance Mediane_{row['Groupe_ID']}"

                # Insérer les données dans les colonnes du nouveau DataFrame 
                new_df.at[0, col1] = row['Duree_min']  
                new_df.at[0, col2] = row['Distance Mediane']
            
            new_df.insert(0, 'cap_ID', unique_val)

            #new_df['Captor'] = unique_val
            
            dfs.append(new_df)
            
        else:
            print("no observations under the cutt-off \n")
            continue
            
        temp_df = pd.concat(dfs, ignore_index=True)
        
    temp_df.insert(0, 'experiment', file_name)
        
    #temp_df['experiment'] = file_name
            
    res.append(temp_df)

    print("================================================================================== \n")
    
final_result = pd.concat(res, ignore_index=True)



experiment_2_2. contact duration 24h_matrice_seed.xlsx is being processed 

Captor  100 is being processed
______________________________________ 

    Groupe_ID  Duree_min  Distance Mediane
0           1       18.0          0.548556
18          2       14.0          1.321643
32          3        7.0          0.834141 

Captor  116 is being processed
______________________________________ 

    Groupe_ID  Duree_min  Distance Mediane
0           1       22.0          1.457987
22          2        9.0          1.636756
31          3        5.0          1.171351 

Captor  255 is being processed
______________________________________ 

   Groupe_ID  Duree_min  Distance Mediane
0          1        5.0          1.733233
5          2        2.0          1.794322
7          3        2.0          1.584297 

Captor  376 is being processed
______________________________________ 

   Groupe_ID  Duree_min  Distance Mediane
0          1        4.0          1.402794
4          2        4.0          1

    Groupe_ID  Duree_min  Distance Mediane
0           1       17.0          0.666127
17          2       16.0          0.639288
33          3       16.0          0.683999 

Captor  436 is being processed
______________________________________ 

    Groupe_ID  Duree_min  Distance Mediane
0           1       13.0          1.753512
13          2       10.0          1.710792
23          3        4.0          1.484331 

Captor  742 is being processed
______________________________________ 

    Groupe_ID  Duree_min  Distance Mediane
0           1       13.0          1.422969
13          2       13.0          1.677697
26          3        8.0          1.797182 

Captor  2372 is being processed
______________________________________ 

    Groupe_ID  Duree_min  Distance Mediane
0           1       25.0          1.265687
25          2       13.0          1.611251
38          3       11.0          1.377060 

Captor  3008 is being processed
______________________________________ 

    Groupe_ID 

In [116]:
identifier_enveloppe("enveloppe1",0.1, 2)

In [117]:
identifier_enveloppe("enveloppe2",0.2, 2)

In [118]:
identifier_enveloppe("enveloppe3",0.3, 2)

In [119]:
identifier_enveloppe("enveloppe4",0.1, 1)

In [120]:
identifier_enveloppe("enveloppe5",0.2, 1)

In [121]:
identifier_enveloppe("enveloppe6",0.3, 1)